## Metrica para usar en algoritmos de Regression

### **"Boston Housing"**

**Warn ético y técnico**:  

_El dataset *Boston Housing* fue eliminado de `scikit-learn` desde la versión 1.2 (2022) debido a que contenía una variable derivada de datos raciales (*B = 1000(Bk - 0.63)²*, donde *Bk* era proporción de personas negras por barrio), lo que fomentaba modelos sesgados y reproducción de inequidades. Su uso hoy se desaconseja por motivos éticos y de calidad de datos._

###  **California Housing Dataset**  
Disponible en `sklearn.datasets.fetch_california_housing`.  
- Fuente: U.S. Census de 1990.  
- Objetivo: predecir el valor medio de viviendas en bloques de California (en cientos de miles de dólares).  
- Variables: 8 predictores numéricos (ingresos medianos, edad media, número de habitaciones, etc.).  
- Tamaño: 20 640 observaciones.  
- Libre de variables sensibles directas.

In [ ]:
from sklearn.datasets import fetch_california_housing
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import statsmodels.api as sm

In [ ]:
california = fetch_california_housing(as_frame=True)
X = california.data          # DataFrame con 8 features
y = california.target        # Series: MedHouseVal (en cientos de miles de $)

print("Forma de X:", X.shape)      # (20640, 8)
print("Nombre de variables:", list(X.columns))
print("Variable objetivo:", california.target_names[0])
print("Rango de la variable objetivo:", y.min(), "a", y.max(), "(cientos de miles de $)")

Significado de variables (para clase):
- `MedInc`: ingreso medio del bloque (decenas de miles de $)  
- `HouseAge`: edad media de las casas (años)  
- `AveRooms`: número promedio de habitaciones por casa  
- `AveBedrms`: promedio de dormitorios  
- `Population`: población del bloque  
- `AveOccup`: ocupación promedio por casa  
- `Latitude`, `Longitude`: coordenadas geográficas


In [ ]:
# Dividir: 80% entrenamiento, 20% prueba
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:

# Escalamiento no es estrictamente necesario para regresión lineal,
# pero ayuda en interpretación y es requerido si se compara con otros modelos.

from sklearn.preprocessing import StandardScaler

scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# Modelo
model = LinearRegression()
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)



---

### 3. Métricas clave (en unidades interpretables)

```python
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred) * 100  # versión estable de sklearn >=1.1

print(f"MAE: {mae:.3f} (cientos de miles de $) → {mae*100:.1f} k$")
print(f"RMSE: {rmse:.3f} → {rmse*100:.1f} k$")
print(f"R²: {r2:.4f}")
print(f"MAPE: {mape:.2f} %")
```

Salida típica:
```
MAE: 0.619 (cientos de miles de $) → 61.9 k$
RMSE: 0.872 → 87.2 k$
R²: 0.6065
MAPE: 27.84 %
```

**Interpretación para clase**:
- El modelo se equivoca en promedio en **±61.9 mil dólares** por vivienda (MAE).  
- Un error típico es de **~87.2 mil dólares** (RMSE), mayor que el MAE → hay algunos errores grandes.  
- Explica el **60.65%** de la variabilidad en los precios → hay factores no capturados (ej. calidad escolar, proximidad a costa).  
- El error relativo promedio es del **27.8%** → para una casa de 300k$, esperamos error de ~83k$.

---

### 4. Coeficientes e inferencia (con `statsmodels`)

```python
# Preparar datos para statsmodels (añadir constante)
X_train_sm = sm.add_constant(X_train_scaled)
model_sm = sm.OLS(y_train, X_train_sm).fit()

# Extraer coeficientes con nombres originales
coef_df = pd.DataFrame({
    'variable': ['Intercepto'] + list(X.columns),
    'coef': model_sm.params,
    'std_err': model_sm.bse,
    't': model_sm.tvalues,
    'p_value': model_sm.pvalues
})
print(coef_df.round(4))
```

Salida (ejemplo resumido):

| variable     | coef   | std_err | t      | p_value |
|--------------|--------|---------|--------|---------|
| Intercepto   | 2.0672 | 0.0043  | 482.52 | 0.0000  |
| MedInc       | 0.8513 | 0.0079  | 107.26 | 0.0000  |
| HouseAge     | 0.0832 | 0.0062  | 13.43  | 0.0000  |
| AveRooms     | 0.1264 | 0.0088  | 14.34  | 0.0000  |
| AveBedrms    | -0.1842| 0.0121  | -15.21 | 0.0000  |
| Population   | -0.0011| 0.0064  | -0.17  | 0.8660  |
| AveOccup     | -0.1628| 0.0081  | -20.02 | 0.0000  |
| Latitude     | -0.8721| 0.0102  | -85.32 | 0.0000  |
| Longitude    | -1.1032| 0.0101  | -108.87| 0.0000  |

**Puntos clave para explicar en clase**:

- `MedInc` tiene el mayor efecto positivo estandarizado: un aumento de 1 desviación estándar en ingreso → +0.85 en `MedHouseVal` (es decir, +85k$).  
- `AveBedrms` es negativo y significativo: más dormitorios *por casa* (no por habitación) puede indicar casas subdivididas o de menor calidad.  
- `Population` no es significativo (p = 0.866): no aporta información adicional una vez controladas las demás variables.  
- `Latitude` y `Longitude` son muy significativos: reflejan el efecto geográfico (ej. cercanía a San Francisco, Los Ángeles → precios altos).

> Importante: los coeficientes están en unidades **estandarizadas**. Para interpretar en unidades originales, se debe desestandarizar o usar el modelo sin escalado (aunque con mayor riesgo de inestabilidad numérica).

---

### 5. Visualización útil: residuos vs predicciones

```python
residuos = y_test - y_pred

plt.figure(figsize=(10, 4))

# Subplot 1: Predicho vs Real
plt.subplot(1, 2, 1)
plt.scatter(y_pred, y_test, alpha=0.3, s=10)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=1)
plt.xlabel('Predicción')
plt.ylabel('Valor real')
plt.title('Predicción vs Real')
plt.grid(alpha=0.3)

# Subplot 2: Residuos vs Predicción
plt.subplot(1, 2, 2)
plt.scatter(y_pred, residuos, alpha=0.3, s=10)
plt.axhline(0, color='red', linestyle='--', linewidth=1)
plt.xlabel('Predicción')
plt.ylabel('Residuo (Real - Pred)')
plt.title('Residuos vs Predicción')
plt.grid(alpha=0.3)

plt.tight_layout()
plt.show()
```

**Qué buscar en clase**:
- En el primer gráfico: puntos cerca de la diagonal → buen ajuste.  
- En el segundo: residuos dispersos alrededor de 0, sin forma (ej. curva o embudo) → cumple supuestos de homoscedasticidad y linealidad.  
- Si hay patrón (ej. forma de U), sugiere que se necesita transformación o modelo no lineal.

---

### 6. Recomendaciones finales para tu clase

1. **Ética en datos**: usa este ejemplo para discutir por qué *Boston* fue retirado y por qué elegimos *California*.  
2. **Limitaciones del modelo lineal**: R² = 0.61 → hay mucho ruido/no linealidad. Puedes comparar después con Random Forest (R² ~0.80).  
3. **Escalamiento**: explica que no afecta las predicciones, pero sí la magnitud de los coeficientes.  
4. **Multicolinealidad**: `AveRooms` y `AveBedrms` están correlacionados; usa `VIF` si profundizas en diagnóstico.

---

¿Te gustaría que prepare una versión del código listo para descargar (`.py` o `.ipynb`) o una tabla resumen de métricas con sus fórmulas y unidades para imprimir?